In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'han'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, False, 15)

In [12]:

if file_parameters['watershed'] == 'nak':
    run_num = range(len(folder))
else:
    run_num = [0]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2013.xlsx', '가평_2014.xlsx', '가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2013.xlsx', '서상_2014.xlsx', '서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2013.xlsx', '의암호_2014.xlsx', '의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2013.xlsx
data/han/자동/가평_2014.xlsx
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-7-31 23:00
data/han/자동/서상_2013.xlsx
data/han/자동/서상_2014.xlsx
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
data/han/자동/서상_2020.xlsx
time range in files :  201

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.8371 - disc_loss: 0.3524 - rmse: 0.4022 - val_loss: 0.5276
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.2628 - disc_loss: 0.3515 - rmse: 0.4463 - val_loss: 0.3924
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.6355 - disc_loss: 0.3607 - rmse: 0.4214 - val_loss: 0.7876
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.6106 - disc_loss: 0.3479 - rmse: 0.4302 - val_loss: 0.4717
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.9754 - disc_loss: 0.3451 - rmse: 0.4345 - val_loss: 0.4171
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.9682 - disc_loss: 0.3524 - rmse: 0.4308 - val_loss: 0.4467
Epoch 55/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 12.6796 - disc_loss: 0.3682 - rmse: 0.4421 - val_loss: 0.3772
Epoch 56/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 11.6739 - disc_loss: 0.2980 - rmse: 0.3806 - val_loss: 0.3434
Epoch 108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3691 - disc_loss: 0.2812 - rmse: 0.3758 - val_loss: 0.4939
Epoch 109/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 8.6863 - disc_loss: 0.2914 - rmse: 0.3700 - val_loss: 0.3303
Epoch 110/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 12.6597 - disc_loss: 0.3025 - rmse: 0.4376 - val_loss: 0.3818
Epoch 111/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 12.0818 - disc_loss: 0.2979 - rmse: 0.3949 - val_loss: 0.3303
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.0748 - disc_loss: 0.2803 - rmse: 0.3903 - val_loss: 0.3185
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.5316 - disc_loss: 0.2839 - rmse: 0.4072 - val_loss: 0.3493
Epoch 114/2000
1/1 [===========

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.7574 - disc_loss: 0.2652 - rmse: 0.3322 - val_loss: 0.3520
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0797 - disc_loss: 0.2538 - rmse: 0.3649 - val_loss: 0.3720
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 19.6666 - disc_loss: 0.2707 - rmse: 0.5825 - val_loss: 0.3838
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.1266 - disc_loss: 0.2628 - rmse: 0.4251 - val_loss: 0.4767
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2642 - disc_loss: 0.2704 - rmse: 0.3213 - val_loss: 0.3867
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.2163 - disc_loss: 0.2604 - rmse: 0.3195 - val_loss: 0.3334
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0850 - disc_loss: 0.2528 - rmse: 0.3398 - val_loss: 0.4483
Epoch 172/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.1317 - disc_loss: 0.2458 - rmse: 0.3980 - val_loss: 0.3206
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6313 - disc_loss: 0.2453 - rmse: 0.3693 - val_loss: 0.5913
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.9016 - disc_loss: 0.2483 - rmse: 0.3540 - val_loss: 0.3923
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6878 - disc_loss: 0.2487 - rmse: 0.3245 - val_loss: 0.3902
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 19.5749 - disc_loss: 0.2343 - rmse: 0.5703 - val_loss: 0.4305
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3338 - disc_loss: 0.2551 - rmse: 0.3840 - val_loss: 0.3187
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6489 - disc_loss: 0.2440 - rmse: 0.3729 - val_loss: 0.3262
Epoch 230/2000
1/1 [==============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6306 - disc_loss: 0.1617 - rmse: 0.3142 - val_loss: 0.3406
Epoch 804/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7043 - disc_loss: 0.1915 - rmse: 0.2985 - val_loss: 0.2999
Epoch 805/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2609 - disc_loss: 0.1720 - rmse: 0.3001 - val_loss: 0.3057
Epoch 806/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5310 - disc_loss: 0.1752 - rmse: 0.3001 - val_loss: 0.4541
Epoch 807/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7115 - disc_loss: 0.1841 - rmse: 0.2862 - val_loss: 0.3572
Epoch 808/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.3277 - disc_loss: 0.1867 - rmse: 0.3346 - val_loss: 0.2930
Epoch 809/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0973 - disc_loss: 0.1678 - rmse: 0.3378 - val_loss: 0.3122
Epoch 810/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6082 - disc_loss: 0.1613 - rmse: 0.3024 - val_loss: 0.9439
Epoch 862/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8859 - disc_loss: 0.1813 - rmse: 0.3164 - val_loss: 0.3375
Epoch 863/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7016 - disc_loss: 0.1770 - rmse: 0.3230 - val_loss: 0.3299
Epoch 864/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6750 - disc_loss: 0.1771 - rmse: 0.3068 - val_loss: 0.3134
Epoch 865/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4029 - disc_loss: 0.1806 - rmse: 0.2779 - val_loss: 0.2979
Epoch 866/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7318 - disc_loss: 0.1781 - rmse: 0.3080 - val_loss: 0.2772
Epoch 867/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3377 - disc_loss: 0.1711 - rmse: 0.3098 - val_loss: 0.3540
Epoch 868/2000
1/1 [===============

1/1 [==============================] - 0s 406ms/step - gen_loss: 4.7885 - disc_loss: 0.1879 - rmse: 0.2838 - val_loss: 0.4121
Epoch 920/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 6.4836 - disc_loss: 0.1776 - rmse: 0.3426 - val_loss: 0.3119
Epoch 921/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.8144 - disc_loss: 0.1678 - rmse: 0.4142 - val_loss: 0.3814
Epoch 922/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2940 - disc_loss: 0.1809 - rmse: 0.2977 - val_loss: 0.3235
Epoch 923/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7325 - disc_loss: 0.1815 - rmse: 0.3097 - val_loss: 0.5558
Epoch 924/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0668 - disc_loss: 0.1815 - rmse: 0.3151 - val_loss: 0.2980
Epoch 925/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9844 - disc_loss: 0.1927 - rmse: 0.3736 - val_loss: 0.4115
Epoch 926/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0440 - disc_loss: 0.1903 - rmse: 0.2984 - val_loss: 0.2821
Epoch 978/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4858 - disc_loss: 0.1668 - rmse: 0.3309 - val_loss: 0.3388
Epoch 979/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3077 - disc_loss: 0.1633 - rmse: 0.2976 - val_loss: 0.3400
Epoch 980/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.7180 - disc_loss: 0.1817 - rmse: 0.9068 - val_loss: 0.2776
Epoch 981/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4311 - disc_loss: 0.1716 - rmse: 0.2902 - val_loss: 0.3627
Epoch 982/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5895 - disc_loss: 0.1853 - rmse: 0.3788 - val_loss: 0.3294
Epoch 983/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 5.6634 - disc_loss: 0.1663 - rmse: 0.3082 - val_loss: 0.3839
Epoch 984/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4188 - disc_loss: 0.1574 - rmse: 0.3400 - val_loss: 0.3707
Epoch 1036/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6650 - disc_loss: 0.1780 - rmse: 0.3263 - val_loss: 0.2618
Epoch 1037/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.8379 - disc_loss: 0.1715 - rmse: 0.2787 - val_loss: 0.3790
Epoch 1038/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3828 - disc_loss: 0.1693 - rmse: 0.2923 - val_loss: 0.3490
Epoch 1039/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6825 - disc_loss: 0.1776 - rmse: 0.2639 - val_loss: 0.2938
Epoch 1040/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.0612 - disc_loss: 0.1748 - rmse: 0.3650 - val_loss: 0.3328
Epoch 1041/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5922 - disc_loss: 0.1697 - rmse: 0.2938 - val_loss: 0.3147
Epoch 1042/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [46]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)



(66456, 36)
toc


In [47]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df, _ = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (46519, 36) val_df.shape :  (13292, 36) test_df.shape: (6645, 36)


In [48]:
#fake_df.shape[0]/8760

In [49]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [50]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [51]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/han/models/han/toc/


In [52]:
rnn_epochs, rnn_in_setps

(15, 240)

## 모델 학습

In [53]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    #training_flag=False, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

## core / window.py / 

In [54]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/han/models/han/toc/
year : 2013 ~ 2020
run :  [0]
target :  toc   4
length :  66456
train=7, test=1, val=2
all_nse :  [0.5774241]
all_pbias :  [-0.8068138]
all_mae :  0.09493463
all_rmse :  0.16117677
all_Rs :  0.5794787232503609
all_R :  0.761235


In [55]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.55391586]
train_pbias :  [-0.98745394]
train_mae :  0.096337155
train_rmse :  0.17067397
train_Rs :  0.5567152092206449
train_R :  0.7461335


In [56]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [0.3949179]
val_pbias :  [1.2445418]
val_mae :  0.11650639
val_rmse :  0.18401681
val_Rs :  0.404545557399306
val_R :  0.63603896


In [57]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.6799347]
test_pbias :  [-0.4307103]
test_mae :  0.07527516
test_rmse :  0.10217622
test_Rs :  0.6858544941779066
test_R :  0.8281633


In [44]:
#train_df.shape, val_df.shape, test_df.shape

In [31]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [24]:
'''
sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 4,
        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1
'''


'\nsub_flag = True\nhistory_nse = []\n\nbob = 0\nb = 1\n\nwhile sub_flag:\n    \n\n    \n    gru_model = model_gru(\n        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,\n        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 4,\n        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)\n    \n    #nse, _, _, _ = compa(\n        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,\n        #predict_day = rnn_predict_day)\n        #predict_day = 4)\n    \n    nse, _, _, _, _, _, _, _ = compa(\n    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,\n    #predict_day = rnn_predict_day)\n    predict_day = 4)\n    \n    if(nse > 0.5):\n        sub_flag = False\n   \n    \n    history_nse.append(nse)\n    \n    print(\'----------------